# SW-10-JSONLD

**Navigation** : [<< 9-SHACL](SW-9-SHACL.ipynb) | [Index](README.md) | [11-RDFStar >>](SW-11-RDFStar.ipynb)

## JSON-LD : Le Web Semantique rencontre JSON

### Duree estimee : 40 minutes

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre JSON-LD et son role de pont entre JSON et le Linked Data
2. Creer et manipuler des contextes JSON-LD (`@context`, `@id`, `@type`, `@graph`)
3. Utiliser le vocabulaire Schema.org pour decrire des entites web
4. Generer et parser des donnees structurees avec rdflib

### Prerequis
- Python 3.10+
- Notebook [SW-8-PythonRDF](SW-8-PythonRDF.ipynb) (bases rdflib et SPARQL)

---

## Installation des dependances

In [1]:
%pip install -q rdflib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


---

## 1. Pourquoi JSON-LD ? Le pont entre JSON et le Linked Data

Le Web regorge de donnees en JSON, le format natif de JavaScript. Mais JSON seul ne porte **aucune semantique** : les cles sont des chaines arbitraires, sans signification universelle. Un champ `"name"` dans une API peut designer un nom de personne, de produit ou de fichier.

**JSON-LD** (JSON for Linking Data) resout ce probleme en ajoutant une couche de contexte (`@context`) qui relie chaque cle JSON a un IRI semantique, transformant du JSON ordinaire en donnees RDF exploitables.

### L'adoption de JSON-LD en chiffres

| Metrique | Valeur | Source |
|----------|--------|--------|
| Part des rich snippets Google utilisant JSON-LD | ~73% | Schema.org community (2024) |
| Amelioration moyenne du CTR avec donnees structurees | +35% | Etudes SEO |
| Format recommande par Google | JSON-LD | Google Developers |
| Types Schema.org disponibles | 800+ | schema.org/docs |

### Historique

JSON-LD a ete cree par le W3C (recommandation JSON-LD 1.0 en 2014, puis 1.1 en 2020). L'objectif etait de rendre le Linked Data accessible aux developpeurs web habitues a JSON, sans leur imposer la syntaxe RDF/XML ou Turtle.

> **Point cle** : JSON-LD est du JSON valide. Tout parser JSON peut le lire. Mais un parser JSON-LD peut en extraire des triples RDF.

In [2]:
import json

# JSON classique : pas de semantique
json_simple = {
    "name": "Ada Lovelace",
    "job": "Mathematician"
}

# JSON-LD : meme structure, mais avec un contexte semantique
jsonld_with_context = {
    "@context": "https://schema.org/",
    "@type": "Person",
    "name": "Ada Lovelace",
    "jobTitle": "Mathematician"
}

print("=== JSON simple (pas de semantique) ===")
print(json.dumps(json_simple, indent=2))
print()
print("=== JSON-LD (avec contexte Schema.org) ===")
print(json.dumps(jsonld_with_context, indent=2))

=== JSON simple (pas de semantique) ===
{
  "name": "Ada Lovelace",
  "job": "Mathematician"
}

=== JSON-LD (avec contexte Schema.org) ===
{
  "@context": "https://schema.org/",
  "@type": "Person",
  "name": "Ada Lovelace",
  "jobTitle": "Mathematician"
}


### Interpretation

Les deux blocs sont du JSON valide, mais seul le second porte une signification universelle :
- `"@context": "https://schema.org/"` declare que les cles utilisent le vocabulaire Schema.org
- `"@type": "Person"` indique qu'il s'agit d'une personne (equivalent de `rdf:type schema:Person`)
- `"name"` est resolu en `schema:name`, une propriete standardisee

Un moteur de recherche comme Google peut donc extraire automatiquement ces informations.

---

## 2. Syntaxe fondamentale de JSON-LD

JSON-LD repose sur quatre mots-cles principaux qui transforment du JSON ordinaire en donnees liees.

### 2.1 `@context` : le dictionnaire de traduction

Le `@context` associe chaque cle JSON a un IRI (Internationalized Resource Identifier). Il peut etre :
- **Une URL** : `"@context": "https://schema.org/"` (contexte distant)
- **Un objet** : definition locale des mappings
- **Un tableau** : combinaison de plusieurs contextes

In [3]:
import json

# Contexte distant (Schema.org)
ctx_remote = {
    "@context": "https://schema.org/",
    "@type": "Person",
    "name": "Alan Turing"
}

# Contexte local (mappings explicites)
ctx_local = {
    "@context": {
        "name": "http://xmlns.com/foaf/0.1/name",
        "homepage": {
            "@id": "http://xmlns.com/foaf/0.1/homepage",
            "@type": "@id"
        }
    },
    "name": "Alan Turing",
    "homepage": "https://en.wikipedia.org/wiki/Alan_Turing"
}

# Contexte combine (tableau)
ctx_combined = {
    "@context": [
        "https://schema.org/",
        {
            "foaf": "http://xmlns.com/foaf/0.1/"
        }
    ],
    "@type": "Person",
    "name": "Alan Turing"
}

print("=== Contexte distant ===")
print(json.dumps(ctx_remote, indent=2))
print()
print("=== Contexte local (mappings explicites) ===")
print(json.dumps(ctx_local, indent=2))
print()
print("=== Contexte combine ===")
print(json.dumps(ctx_combined, indent=2))

=== Contexte distant ===
{
  "@context": "https://schema.org/",
  "@type": "Person",
  "name": "Alan Turing"
}

=== Contexte local (mappings explicites) ===
{
  "@context": {
    "name": "http://xmlns.com/foaf/0.1/name",
    "homepage": {
      "@id": "http://xmlns.com/foaf/0.1/homepage",
      "@type": "@id"
    }
  },
  "name": "Alan Turing",
  "homepage": "https://en.wikipedia.org/wiki/Alan_Turing"
}

=== Contexte combine ===
{
  "@context": [
    "https://schema.org/",
    {
      "foaf": "http://xmlns.com/foaf/0.1/"
    }
  ],
  "@type": "Person",
  "name": "Alan Turing"
}


### 2.2 `@id` et `@type` : identifier et typer les ressources

- **`@id`** : identifie la ressource (le sujet du triple RDF). Si absent, un blank node est cree.
- **`@type`** : definit le type de la ressource (equivalent de `rdf:type`).

In [4]:
import json

person_with_id = {
    "@context": "https://schema.org/",
    "@id": "https://example.org/people/turing",
    "@type": "Person",
    "name": "Alan Turing",
    "birthDate": "1912-06-23",
    "birthPlace": {
        "@type": "Place",
        "name": "London"
    }
}

print(json.dumps(person_with_id, indent=2))
print()
print("Triples RDF generes :")
print("  <https://example.org/people/turing> rdf:type schema:Person .")
print('  <https://example.org/people/turing> schema:name "Alan Turing" .')
print('  <https://example.org/people/turing> schema:birthDate "1912-06-23" .')
print("  <https://example.org/people/turing> schema:birthPlace _:b0 .")
print("  _:b0 rdf:type schema:Place .")
print('  _:b0 schema:name "London" .')

{
  "@context": "https://schema.org/",
  "@id": "https://example.org/people/turing",
  "@type": "Person",
  "name": "Alan Turing",
  "birthDate": "1912-06-23",
  "birthPlace": {
    "@type": "Place",
    "name": "London"
  }
}

Triples RDF generes :
  <https://example.org/people/turing> rdf:type schema:Person .
  <https://example.org/people/turing> schema:name "Alan Turing" .
  <https://example.org/people/turing> schema:birthDate "1912-06-23" .
  <https://example.org/people/turing> schema:birthPlace _:b0 .
  _:b0 rdf:type schema:Place .
  _:b0 schema:name "London" .


### 2.3 `@graph` : regrouper plusieurs ressources

Le mot-cle `@graph` permet de decrire plusieurs entites dans un seul document JSON-LD, partageant le meme `@context`.

In [5]:
import json

multi_entities = {
    "@context": "https://schema.org/",
    "@graph": [
        {
            "@type": "Person",
            "@id": "https://example.org/people/turing",
            "name": "Alan Turing",
            "alumniOf": {"@id": "https://example.org/org/cambridge"}
        },
        {
            "@type": "CollegeOrUniversity",
            "@id": "https://example.org/org/cambridge",
            "name": "University of Cambridge",
            "foundingDate": "1209"
        },
        {
            "@type": "Person",
            "@id": "https://example.org/people/lovelace",
            "name": "Ada Lovelace",
            "jobTitle": "Mathematician"
        }
    ]
}

print(f"Document contenant {len(multi_entities['@graph'])} entites :")
for entity in multi_entities["@graph"]:
    print(f"  - {entity['@type']} : {entity['name']} ({entity['@id']})")

Document contenant 3 entites :
  - Person : Alan Turing (https://example.org/people/turing)
  - CollegeOrUniversity : University of Cambridge (https://example.org/org/cambridge)
  - Person : Ada Lovelace (https://example.org/people/lovelace)


### 2.4 Formes JSON-LD : Compacte, Etendue et Aplatie

Un meme graphe RDF peut etre represente sous trois formes JSON-LD :

| Forme | Description | Usage |
|-------|-------------|-------|
| **Compacte** | Utilise un `@context` pour abreger les IRIs | Production web (lisibilite) |
| **Etendue** | IRIs complets, pas de `@context` | Echanges machine a machine |
| **Aplatie** | Toutes les entites au meme niveau dans `@graph` | Normalisation, deduplication |

In [6]:
import json

# Forme COMPACTE (la plus courante)
compact_form = {
    "@context": "https://schema.org/",
    "@type": "Person",
    "name": "Marie Curie",
    "jobTitle": "Physicist"
}

# Forme ETENDUE (expanded) - IRIs complets, pas de contexte
expanded_form = [
    {
        "@type": ["http://schema.org/Person"],
        "http://schema.org/name": [
            {"@value": "Marie Curie"}
        ],
        "http://schema.org/jobTitle": [
            {"@value": "Physicist"}
        ]
    }
]

# Forme APLATIE (flattened) - toutes les entites au meme niveau
flattened_form = {
    "@context": "https://schema.org/",
    "@graph": [
        {
            "@id": "_:b0",
            "@type": "Person",
            "name": "Marie Curie",
            "jobTitle": "Physicist"
        }
    ]
}

print("=== Forme COMPACTE ===")
print(json.dumps(compact_form, indent=2))
print()
print("=== Forme ETENDUE (expanded) ===")
print(json.dumps(expanded_form, indent=2))
print()
print("=== Forme APLATIE (flattened) ===")
print(json.dumps(flattened_form, indent=2))

=== Forme COMPACTE ===
{
  "@context": "https://schema.org/",
  "@type": "Person",
  "name": "Marie Curie",
  "jobTitle": "Physicist"
}

=== Forme ETENDUE (expanded) ===
[
  {
    "@type": [
      "http://schema.org/Person"
    ],
    "http://schema.org/name": [
      {
        "@value": "Marie Curie"
      }
    ],
    "http://schema.org/jobTitle": [
      {
        "@value": "Physicist"
      }
    ]
  }
]

=== Forme APLATIE (flattened) ===
{
  "@context": "https://schema.org/",
  "@graph": [
    {
      "@id": "_:b0",
      "@type": "Person",
      "name": "Marie Curie",
      "jobTitle": "Physicist"
    }
  ]
}


### Interpretation

Les trois formes representent exactement les memes triples RDF. La forme **compacte** est la plus utilisee dans les pages web car elle est lisible par les developpeurs. La forme **etendue** est utile pour le traitement automatise (pas besoin de resoudre le contexte). La forme **aplatie** normalise la structure pour faciliter la comparaison et la fusion de documents.

---

## 3. Le vocabulaire Schema.org

**Schema.org** est le vocabulaire collaboratif cree par Google, Microsoft, Yahoo et Yandex pour structurer les donnees du Web. Il definit plus de 800 types et 1500 proprietes couvrant les domaines les plus courants.

### Types les plus utilises

| Type | Description | Exemple d'usage |
|------|-------------|----------------|
| `Person` | Personne physique | Profil, auteur d'article |
| `Organization` | Entreprise, association | Page entreprise |
| `Product` | Produit commercial | Fiche produit e-commerce |
| `Event` | Evenement | Concert, conference |
| `Article` | Article de presse/blog | Billet de blog |
| `Recipe` | Recette de cuisine | Site culinaire |
| `FAQPage` | Page de FAQ | Support client |
| `BreadcrumbList` | Fil d'Ariane | Navigation de site |
| `Course` | Cours en ligne | Plateforme educative |

### 3.1 Explorer le namespace Schema.org avec rdflib

rdflib fournit un namespace `SDO` (Schema.org) predefini pour acceder aux termes Schema.org.

In [7]:
from rdflib import Namespace, URIRef
from rdflib.namespace import RDF, RDFS

# Definir le namespace Schema.org
SDO = Namespace("https://schema.org/")

# Exemples de termes Schema.org
common_types = [
    ("Person", SDO.Person),
    ("Organization", SDO.Organization),
    ("Product", SDO.Product),
    ("Event", SDO.Event),
    ("Article", SDO.Article),
    ("Course", SDO.Course),
]

common_props = [
    ("name", SDO.name),
    ("description", SDO.description),
    ("url", SDO.url),
    ("author", SDO.author),
    ("datePublished", SDO.datePublished),
]

print("=== Types Schema.org courants ===")
for label, uri in common_types:
    print(f"  {label:20s} -> {uri}")

print()
print("=== Proprietes Schema.org courantes ===")
for label, uri in common_props:
    print(f"  {label:20s} -> {uri}")

=== Types Schema.org courants ===
  Person               -> https://schema.org/Person
  Organization         -> https://schema.org/Organization
  Product              -> https://schema.org/Product
  Event                -> https://schema.org/Event
  Article              -> https://schema.org/Article
  Course               -> https://schema.org/Course

=== Proprietes Schema.org courantes ===
  name                 -> https://schema.org/name
  description          -> https://schema.org/description
  url                  -> https://schema.org/url
  author               -> https://schema.org/author
  datePublished        -> https://schema.org/datePublished


### 3.2 Charger et afficher le fichier `data/product.jsonld`

Le fichier `data/product.jsonld` contient un exemple Schema.org de type **Product** decrivant le cours CoursIA. Chargeons-le et examinons sa structure.

In [8]:
import json
from pathlib import Path

# Charger le fichier JSON-LD
product_path = Path("data/product.jsonld")
with open(product_path, "r", encoding="utf-8") as f:
    product_data = json.load(f)

print("=== Contenu de data/product.jsonld ===")
print(json.dumps(product_data, indent=2, ensure_ascii=False))
print()
print(f"Type : {product_data['@type']}")
print(f"Nom  : {product_data['name']}")
print(f"Auteur : {product_data['author']['name']}")
print(f"Sujets enseignes : {len(product_data['teaches'])}")
for i, topic in enumerate(product_data["teaches"], 1):
    print(f"  {i}. {topic}")

=== Contenu de data/product.jsonld ===
{
  "@context": "https://schema.org/",
  "@type": "Product",
  "name": "Semantic Web avec Python et .NET",
  "description": "Cours complet sur le Web Semantique, de RDF aux graphes de connaissances",
  "image": "https://example.org/images/semantic-web-course.png",
  "brand": {
    "@type": "Brand",
    "name": "CoursIA"
  },
  "offers": {
    "@type": "Offer",
    "price": "0",
    "priceCurrency": "EUR",
    "availability": "https://schema.org/InStock",
    "url": "https://github.com/jsboige/CoursIA"
  },
  "educationalLevel": "University",
  "teaches": [
    "RDF et triples",
    "SPARQL queries",
    "OWL ontologies",
    "SHACL validation",
    "JSON-LD et Schema.org",
    "Knowledge Graphs",
    "GraphRAG"
  ],
  "author": {
    "@type": "Person",
    "name": "Jean-Sebastien Boige"
  }
}

Type : Product
Nom  : Semantic Web avec Python et .NET
Auteur : Jean-Sebastien Boige
Sujets enseignes : 7
  1. RDF et triples
  2. SPARQL queries
  3. OWL o

### Interpretation : Schema.org Product

Ce document JSON-LD decrit un produit educatif avec les proprietes Schema.org suivantes :

| Propriete | Valeur | Signification Schema.org |
|-----------|--------|-------------------------|
| `@type` | Product | Type de l'entite |
| `name` | Semantic Web avec Python et .NET | Nom du produit |
| `brand` | CoursIA | Marque associee |
| `offers` | Prix 0 EUR, InStock | Disponibilite et prix |
| `educationalLevel` | University | Niveau educatif |
| `teaches` | 7 sujets | Competences enseignees |
| `author` | Jean-Sebastien Boige | Auteur du cours |

> **Note** : Ce JSON-LD pourrait etre integre directement dans une balise `<script type="application/ld+json">` d'une page HTML pour que Google affiche un rich snippet.

---

## 4. Creer du JSON-LD avec rdflib

rdflib permet de construire un graphe RDF et de le serialiser directement en JSON-LD. C'est l'approche programmatique pour generer des donnees structurees.

### 4.1 Construire un graphe et serialiser en JSON-LD

Nous allons creer une entite Schema.org Person avec plusieurs proprietes, puis l'exporter en JSON-LD.

In [9]:
from rdflib import Graph, Literal, Namespace, URIRef, BNode
from rdflib.namespace import RDF, RDFS, XSD

# Namespaces
SDO = Namespace("https://schema.org/")
EX = Namespace("https://example.org/people/")

# Creer un graphe
g = Graph()
g.bind("schema", SDO)
g.bind("ex", EX)

# Definir une personne
person = EX.ada_lovelace

g.add((person, RDF.type, SDO.Person))
g.add((person, SDO.name, Literal("Ada Lovelace")))
g.add((person, SDO.jobTitle, Literal("Mathematician and Writer")))
g.add((person, SDO.email, Literal("ada@example.org")))
g.add((person, SDO.birthDate, Literal("1815-12-10", datatype=XSD.date)))

# Ajouter une relation "knows"
babbage = EX.charles_babbage
g.add((babbage, RDF.type, SDO.Person))
g.add((babbage, SDO.name, Literal("Charles Babbage")))
g.add((babbage, SDO.jobTitle, Literal("Inventor")))
g.add((person, SDO.knows, babbage))

print(f"Graphe construit : {len(g)} triples")
print()

# Serialiser en JSON-LD
jsonld_output = g.serialize(format="json-ld", indent=2)
print("=== Serialisation JSON-LD ===")
print(jsonld_output)

Graphe construit : 9 triples

=== Serialisation JSON-LD ===
[
  {
    "@id": "https://example.org/people/ada_lovelace",
    "@type": [
      "https://schema.org/Person"
    ],
    "https://schema.org/birthDate": [
      {
        "@type": "http://www.w3.org/2001/XMLSchema#date",
        "@value": "1815-12-10"
      }
    ],
    "https://schema.org/email": [
      {
        "@value": "ada@example.org"
      }
    ],
    "https://schema.org/jobTitle": [
      {
        "@value": "Mathematician and Writer"
      }
    ],
    "https://schema.org/knows": [
      {
        "@id": "https://example.org/people/charles_babbage"
      }
    ],
    "https://schema.org/name": [
      {
        "@value": "Ada Lovelace"
      }
    ]
  },
  {
    "@id": "https://example.org/people/charles_babbage",
    "@type": [
      "https://schema.org/Person"
    ],
    "https://schema.org/jobTitle": [
      {
        "@value": "Inventor"
      }
    ],
    "https://schema.org/name": [
      {
        "@value": "

### Interpretation

rdflib genere automatiquement le JSON-LD a partir du graphe. Notons que :
- Les URIs sont expansees en IRIs complets (forme etendue par defaut)
- Les types de donnees (`xsd:date`) sont preserves
- La relation `knows` lie deux ressources par leur `@id`

> **Note technique** : Pour obtenir la forme compacte avec un `@context`, il faut passer un contexte au serialiseur. Nous verrons cela dans un exercice.

### 4.2 Serialisation compacte avec contexte

Pour generer du JSON-LD compact lisible, on peut fournir un contexte de serialisation.

In [10]:
import json

# Definir le contexte pour la serialisation compacte
context = {
    "schema": "https://schema.org/",
    "ex": "https://example.org/people/",
    "name": "schema:name",
    "jobTitle": "schema:jobTitle",
    "email": "schema:email",
    "birthDate": "schema:birthDate",
    "knows": {"@id": "schema:knows", "@type": "@id"}
}

# Serialiser avec le contexte
jsonld_compact = g.serialize(
    format="json-ld",
    indent=2,
    context=context
)

print("=== JSON-LD compact avec contexte ===")
print(jsonld_compact)

=== JSON-LD compact avec contexte ===
{
  "@context": {
    "birthDate": "schema:birthDate",
    "email": "schema:email",
    "ex": "https://example.org/people/",
    "jobTitle": "schema:jobTitle",
    "knows": {
      "@id": "schema:knows",
      "@type": "@id"
    },
    "name": "schema:name",
    "schema": "https://schema.org/"
  },
  "@graph": [
    {
      "@id": "ex:ada_lovelace",
      "@type": "schema:Person",
      "birthDate": {
        "@type": "http://www.w3.org/2001/XMLSchema#date",
        "@value": "1815-12-10"
      },
      "email": "ada@example.org",
      "jobTitle": "Mathematician and Writer",
      "knows": "ex:charles_babbage",
      "name": "Ada Lovelace"
    },
    {
      "@id": "ex:charles_babbage",
      "@type": "schema:Person",
      "jobTitle": "Inventor",
      "name": "Charles Babbage"
    }
  ]
}


---

## 5. Parser du JSON-LD

L'operation inverse est tout aussi importante : charger du JSON-LD dans un graphe rdflib pour l'interroger avec SPARQL ou le convertir dans d'autres formats.

### 5.1 Charger une chaine JSON-LD dans rdflib

In [11]:
from rdflib import Graph
import json

# Document JSON-LD a parser
jsonld_string = json.dumps({
    "@context": "https://schema.org/",
    "@id": "https://example.org/events/pycon2025",
    "@type": "Event",
    "name": "PyCon France 2025",
    "startDate": "2025-10-23",
    "endDate": "2025-10-26",
    "location": {
        "@type": "Place",
        "name": "Strasbourg",
        "address": "Palais de la Musique et des Congres"
    },
    "organizer": {
        "@type": "Organization",
        "name": "AFPy"
    }
})

# Parser dans un graphe
g2 = Graph()
g2.parse(data=jsonld_string, format="json-ld")

print(f"Triples charges : {len(g2)}")
print()
print("=== Tous les triples ===")
for s, p, o in g2:
    # Raccourcir les URIs pour la lisibilite
    s_short = str(s).replace("https://schema.org/", "schema:").replace("https://example.org/events/", "ex:")
    p_short = str(p).replace("https://schema.org/", "schema:").replace("http://www.w3.org/1999/02/22-rdf-syntax-ns#", "rdf:")
    o_short = str(o).replace("https://schema.org/", "schema:").replace("https://example.org/events/", "ex:")
    print(f"  {s_short}  {p_short}  {o_short}")

Triples charges : 11

=== Tous les triples ===
  ex:pycon2025  http://schema.org/name  PyCon France 2025
  N52a243d90ee4485d9363a0eb38d2b789  http://schema.org/name  AFPy
  N5567792fb9724e2482671f2b66ddb6a1  http://schema.org/address  Palais de la Musique et des Congres
  ex:pycon2025  http://schema.org/startDate  2025-10-23
  ex:pycon2025  http://schema.org/endDate  2025-10-26
  N5567792fb9724e2482671f2b66ddb6a1  http://schema.org/name  Strasbourg
  N52a243d90ee4485d9363a0eb38d2b789  rdf:type  http://schema.org/Organization
  N5567792fb9724e2482671f2b66ddb6a1  rdf:type  http://schema.org/Place
  ex:pycon2025  http://schema.org/organizer  N52a243d90ee4485d9363a0eb38d2b789
  ex:pycon2025  rdf:type  http://schema.org/Event
  ex:pycon2025  http://schema.org/location  N5567792fb9724e2482671f2b66ddb6a1


### 5.2 Interroger avec SPARQL

Une fois le JSON-LD charge dans un graphe, nous pouvons utiliser toute la puissance de SPARQL.

In [12]:
# Requete SPARQL sur le graphe charge depuis JSON-LD
query = """
PREFIX schema: <https://schema.org/>

SELECT ?name ?type ?startDate
WHERE {
    ?entity schema:name ?name .
    ?entity a ?type .
    OPTIONAL { ?entity schema:startDate ?startDate }
}
ORDER BY ?name
"""

print("=== Resultats SPARQL ===")
print(f"{'Nom':<35} {'Type':<25} {'Date debut'}")
print("-" * 75)

for row in g2.query(query):
    name = str(row.name)
    rdf_type = str(row.type).replace("https://schema.org/", "schema:")
    start = str(row.startDate) if row.startDate else "-"
    print(f"{name:<35} {rdf_type:<25} {start}")

=== Resultats SPARQL ===
Nom                                 Type                      Date debut
---------------------------------------------------------------------------


### 5.3 Convertir en Turtle

rdflib permet de convertir le JSON-LD en n'importe quel format RDF, notamment Turtle qui est plus lisible pour les humains.

In [13]:
# Convertir le graphe JSON-LD en Turtle
turtle_output = g2.serialize(format="turtle")
print("=== Conversion JSON-LD -> Turtle ===")
print(turtle_output)

=== Conversion JSON-LD -> Turtle ===
@prefix schema1: <http://schema.org/> .

<https://example.org/events/pycon2025> a schema1:Event ;
    schema1:endDate "2025-10-26"^^schema1:Date ;
    schema1:location [ a schema1:Place ;
            schema1:address "Palais de la Musique et des Congres" ;
            schema1:name "Strasbourg" ] ;
    schema1:name "PyCon France 2025" ;
    schema1:organizer [ a schema1:Organization ;
            schema1:name "AFPy" ] ;
    schema1:startDate "2025-10-23"^^schema1:Date .




### 5.4 Round-trip : JSON-LD -> Turtle -> JSON-LD

Verifions que la conversion est sans perte : JSON-LD vers Turtle, puis retour en JSON-LD.

In [14]:
from rdflib import Graph

# Etape 1 : On a deja g2 (charge depuis JSON-LD)
triples_original = len(g2)

# Etape 2 : Serialiser en Turtle
turtle_str = g2.serialize(format="turtle")

# Etape 3 : Re-parser le Turtle
g3 = Graph()
g3.parse(data=turtle_str, format="turtle")
triples_after_turtle = len(g3)

# Etape 4 : Re-serialiser en JSON-LD
jsonld_roundtrip = g3.serialize(format="json-ld", indent=2)

# Etape 5 : Re-parser le JSON-LD
g4 = Graph()
g4.parse(data=jsonld_roundtrip, format="json-ld")
triples_final = len(g4)

print("=== Verification du round-trip ===")
print(f"Triples apres JSON-LD initial   : {triples_original}")
print(f"Triples apres Turtle            : {triples_after_turtle}")
print(f"Triples apres JSON-LD (retour)  : {triples_final}")
print()

# Verifier l'isomorphisme
is_same = triples_original == triples_final
print(f"Conservation des triples : {'OUI' if is_same else 'NON'} ({triples_original} -> {triples_final})")
print()
print("=== JSON-LD final (apres round-trip) ===")
print(jsonld_roundtrip)

=== Verification du round-trip ===
Triples apres JSON-LD initial   : 11
Triples apres Turtle            : 11
Triples apres JSON-LD (retour)  : 11

Conservation des triples : OUI (11 -> 11)

=== JSON-LD final (apres round-trip) ===
[
  {
    "@id": "https://example.org/events/pycon2025",
    "@type": [
      "http://schema.org/Event"
    ],
    "http://schema.org/endDate": [
      {
        "@type": "http://schema.org/Date",
        "@value": "2025-10-26"
      }
    ],
    "http://schema.org/location": [
      {
        "@id": "_:nccd4aebd0e974beea9d6ddd433d4f9e4b1"
      }
    ],
    "http://schema.org/name": [
      {
        "@value": "PyCon France 2025"
      }
    ],
    "http://schema.org/organizer": [
      {
        "@id": "_:nccd4aebd0e974beea9d6ddd433d4f9e4b2"
      }
    ],
    "http://schema.org/startDate": [
      {
        "@type": "http://schema.org/Date",
        "@value": "2025-10-23"
      }
    ]
  },
  {
    "@id": "_:nccd4aebd0e974beea9d6ddd433d4f9e4b1",
    "@type

### Interpretation : Round-trip

Le nombre de triples est conserve a travers le cycle JSON-LD -> Turtle -> JSON-LD. Cela confirme que rdflib effectue une conversion fidele entre les formats. Les seules differences peuvent etre cosmetiques :
- L'ordre des proprietes peut changer
- Les identifiants de blank nodes peuvent etre renommes
- La structure d'imbrication peut varier (aplati vs imbrique)

Mais les triples RDF sous-jacents sont identiques.

---

## 6. Cas d'usage web pratiques

JSON-LD est principalement utilise pour integrer des **donnees structurees** dans les pages HTML, permettant aux moteurs de recherche d'afficher des **rich snippets** (resultats enrichis). Voici les schemas les plus courants.

### 6.1 Recipe (Recette de cuisine)

Le schema `Recipe` est l'un des plus utilises. Il permet a Google d'afficher des cartes de recettes avec image, temps de preparation et notes.

In [15]:
import json

recipe = {
    "@context": "https://schema.org/",
    "@type": "Recipe",
    "name": "Ratatouille Provencale",
    "author": {
        "@type": "Person",
        "name": "Chef Marie"
    },
    "datePublished": "2024-06-15",
    "description": "Recette traditionnelle de ratatouille avec legumes frais du marche.",
    "prepTime": "PT30M",
    "cookTime": "PT45M",
    "totalTime": "PT1H15M",
    "recipeYield": "4 portions",
    "recipeCategory": "Plat principal",
    "recipeCuisine": "Francaise",
    "recipeIngredient": [
        "2 aubergines",
        "3 courgettes",
        "2 poivrons rouges",
        "4 tomates",
        "2 oignons",
        "3 gousses d'ail",
        "Huile d'olive",
        "Herbes de Provence"
    ],
    "recipeInstructions": [
        {
            "@type": "HowToStep",
            "text": "Couper tous les legumes en des de taille similaire."
        },
        {
            "@type": "HowToStep",
            "text": "Faire revenir les oignons et l'ail dans l'huile d'olive."
        },
        {
            "@type": "HowToStep",
            "text": "Ajouter les legumes par ordre de fermete, cuire 45 minutes a feu doux."
        }
    ],
    "nutrition": {
        "@type": "NutritionInformation",
        "calories": "180 calories",
        "fatContent": "8 g"
    },
    "aggregateRating": {
        "@type": "AggregateRating",
        "ratingValue": "4.7",
        "ratingCount": "342"
    }
}

print("=== Schema.org Recipe ===")
print(json.dumps(recipe, indent=2, ensure_ascii=False))
print()
print(f"Recette : {recipe['name']}")
print(f"Preparation : {recipe['prepTime']} | Cuisson : {recipe['cookTime']}")
print(f"Ingredients : {len(recipe['recipeIngredient'])}")
print(f"Etapes : {len(recipe['recipeInstructions'])}")
print(f"Note : {recipe['aggregateRating']['ratingValue']}/5 ({recipe['aggregateRating']['ratingCount']} avis)")

=== Schema.org Recipe ===
{
  "@context": "https://schema.org/",
  "@type": "Recipe",
  "name": "Ratatouille Provencale",
  "author": {
    "@type": "Person",
    "name": "Chef Marie"
  },
  "datePublished": "2024-06-15",
  "description": "Recette traditionnelle de ratatouille avec legumes frais du marche.",
  "prepTime": "PT30M",
  "cookTime": "PT45M",
  "totalTime": "PT1H15M",
  "recipeYield": "4 portions",
  "recipeCategory": "Plat principal",
  "recipeCuisine": "Francaise",
  "recipeIngredient": [
    "2 aubergines",
    "3 courgettes",
    "2 poivrons rouges",
    "4 tomates",
    "2 oignons",
    "3 gousses d'ail",
    "Huile d'olive",
    "Herbes de Provence"
  ],
  "recipeInstructions": [
    {
      "@type": "HowToStep",
      "text": "Couper tous les legumes en des de taille similaire."
    },
    {
      "@type": "HowToStep",
      "text": "Faire revenir les oignons et l'ail dans l'huile d'olive."
    },
    {
      "@type": "HowToStep",
      "text": "Ajouter les legumes pa

### 6.2 Event (Evenement)

Le schema `Event` permet l'affichage de cartes evenements dans les resultats de recherche.

In [16]:
import json

event = {
    "@context": "https://schema.org/",
    "@type": "Event",
    "name": "Conference Web Semantique 2025",
    "description": "Journee consacree aux technologies du Web Semantique et aux graphes de connaissances.",
    "startDate": "2025-11-15T09:00:00+01:00",
    "endDate": "2025-11-15T18:00:00+01:00",
    "eventAttendanceMode": "https://schema.org/MixedEventAttendanceMode",
    "eventStatus": "https://schema.org/EventScheduled",
    "location": {
        "@type": "Place",
        "name": "Amphitheatre Gaston Berger",
        "address": {
            "@type": "PostalAddress",
            "streetAddress": "Campus Universitaire",
            "addressLocality": "Lyon",
            "postalCode": "69007",
            "addressCountry": "FR"
        }
    },
    "organizer": {
        "@type": "Organization",
        "name": "Association Web Semantique France",
        "url": "https://example.org/awsf"
    },
    "offers": {
        "@type": "Offer",
        "price": "50",
        "priceCurrency": "EUR",
        "availability": "https://schema.org/InStock",
        "validFrom": "2025-06-01"
    }
}

print("=== Schema.org Event ===")
print(json.dumps(event, indent=2, ensure_ascii=False))

=== Schema.org Event ===
{
  "@context": "https://schema.org/",
  "@type": "Event",
  "name": "Conference Web Semantique 2025",
  "description": "Journee consacree aux technologies du Web Semantique et aux graphes de connaissances.",
  "startDate": "2025-11-15T09:00:00+01:00",
  "endDate": "2025-11-15T18:00:00+01:00",
  "eventAttendanceMode": "https://schema.org/MixedEventAttendanceMode",
  "eventStatus": "https://schema.org/EventScheduled",
  "location": {
    "@type": "Place",
    "name": "Amphitheatre Gaston Berger",
    "address": {
      "@type": "PostalAddress",
      "streetAddress": "Campus Universitaire",
      "addressLocality": "Lyon",
      "postalCode": "69007",
      "addressCountry": "FR"
    }
  },
  "organizer": {
    "@type": "Organization",
    "name": "Association Web Semantique France",
    "url": "https://example.org/awsf"
  },
  "offers": {
    "@type": "Offer",
    "price": "50",
    "priceCurrency": "EUR",
    "availability": "https://schema.org/InStock",
    "

### 6.3 BreadcrumbList et FAQPage

Deux schemas tres utilises pour ameliorer l'affichage dans les SERP (Search Engine Results Pages) :
- **BreadcrumbList** : affiche le fil d'Ariane dans les resultats
- **FAQPage** : affiche des questions/reponses depliables

In [17]:
import json

# BreadcrumbList - fil d'Ariane
breadcrumb = {
    "@context": "https://schema.org/",
    "@type": "BreadcrumbList",
    "itemListElement": [
        {
            "@type": "ListItem",
            "position": 1,
            "name": "Accueil",
            "item": "https://example.org/"
        },
        {
            "@type": "ListItem",
            "position": 2,
            "name": "Cours IA",
            "item": "https://example.org/cours-ia/"
        },
        {
            "@type": "ListItem",
            "position": 3,
            "name": "Web Semantique",
            "item": "https://example.org/cours-ia/semantic-web/"
        }
    ]
}

# FAQPage - questions frequentes
faq = {
    "@context": "https://schema.org/",
    "@type": "FAQPage",
    "mainEntity": [
        {
            "@type": "Question",
            "name": "Qu'est-ce que JSON-LD ?",
            "acceptedAnswer": {
                "@type": "Answer",
                "text": "JSON-LD (JSON for Linking Data) est un format W3C qui ajoute une couche semantique au JSON, permettant de representer des donnees RDF dans un format familier aux developpeurs web."
            }
        },
        {
            "@type": "Question",
            "name": "Pourquoi utiliser Schema.org ?",
            "acceptedAnswer": {
                "@type": "Answer",
                "text": "Schema.org est le vocabulaire standard recommande par Google, Microsoft et Yahoo pour structurer les donnees des pages web. Il permet d'obtenir des rich snippets dans les resultats de recherche."
            }
        },
        {
            "@type": "Question",
            "name": "JSON-LD remplace-t-il Turtle ou RDF/XML ?",
            "acceptedAnswer": {
                "@type": "Answer",
                "text": "Non, JSON-LD est une serialisation complementaire de RDF. Turtle reste prefere pour l'edition humaine et les ontologies, tandis que JSON-LD excelle dans l'integration web."
            }
        }
    ]
}

print("=== BreadcrumbList (fil d'Ariane) ===")
for item in breadcrumb["itemListElement"]:
    print(f"  {item['position']}. {item['name']} -> {item['item']}")

print()
print("=== FAQPage (questions frequentes) ===")
for qa in faq["mainEntity"]:
    print(f"  Q: {qa['name']}")
    print(f"  R: {qa['acceptedAnswer']['text'][:80]}...")
    print()

=== BreadcrumbList (fil d'Ariane) ===
  1. Accueil -> https://example.org/
  2. Cours IA -> https://example.org/cours-ia/
  3. Web Semantique -> https://example.org/cours-ia/semantic-web/

=== FAQPage (questions frequentes) ===
  Q: Qu'est-ce que JSON-LD ?
  R: JSON-LD (JSON for Linking Data) est un format W3C qui ajoute une couche semantiq...

  Q: Pourquoi utiliser Schema.org ?
  R: Schema.org est le vocabulaire standard recommande par Google, Microsoft et Yahoo...

  Q: JSON-LD remplace-t-il Turtle ou RDF/XML ?
  R: Non, JSON-LD est une serialisation complementaire de RDF. Turtle reste prefere p...



### Interpretation : cas d'usage web

Ces schemas sont integres dans les pages HTML via la balise :

```html
<script type="application/ld+json">
{ ... le JSON-LD ici ... }
</script>
```

| Schema | Rich snippet Google | Impact SEO |
|--------|-------------------|------------|
| Recipe | Carte avec image, temps, note | Fort (carousels recettes) |
| Event | Date, lieu, prix | Moyen a fort |
| BreadcrumbList | Chemin hierarchique dans les SERP | Moyen |
| FAQPage | Questions depliables sous le resultat | Fort (visibilite accrue) |

> **Conseil pratique** : Utilisez l'outil [Google Rich Results Test](https://search.google.com/test/rich-results) pour valider vos schemas avant deploiement.

---

## 7. JSON-LD vs autres formats RDF

RDF peut etre serialise dans plusieurs formats. Chaque format a ses forces et ses faiblesses selon le contexte d'utilisation.

In [18]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, XSD

# Creer un petit graphe de reference pour la comparaison
SDO = Namespace("https://schema.org/")
g_compare = Graph()
g_compare.bind("schema", SDO)

person_uri = URIRef("https://example.org/people/curie")
g_compare.add((person_uri, RDF.type, SDO.Person))
g_compare.add((person_uri, SDO.name, Literal("Marie Curie")))
g_compare.add((person_uri, SDO.birthDate, Literal("1867-11-07", datatype=XSD.date)))

print(f"Graphe de reference : {len(g_compare)} triples")
print()

# Serialiser dans les 4 formats
formats = [
    ("JSON-LD", "json-ld"),
    ("Turtle", "turtle"),
    ("RDF/XML", "xml"),
    ("N-Triples", "nt"),
]

for label, fmt in formats:
    output = g_compare.serialize(format=fmt)
    lines = [l for l in output.strip().split("\n") if l.strip()]
    size = len(output.encode("utf-8"))
    print(f"{'=' * 60}")
    print(f"=== {label} ({size} octets, {len(lines)} lignes) ===")
    print(f"{'=' * 60}")
    print(output.strip())
    print()

Graphe de reference : 3 triples

=== JSON-LD (351 octets, 19 lignes) ===
[
  {
    "@id": "https://example.org/people/curie",
    "@type": [
      "https://schema.org/Person"
    ],
    "https://schema.org/birthDate": [
      {
        "@type": "http://www.w3.org/2001/XMLSchema#date",
        "@value": "1867-11-07"
      }
    ],
    "https://schema.org/name": [
      {
        "@value": "Marie Curie"
      }
    ]
  }
]

=== Turtle (224 octets, 5 lignes) ===
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://example.org/people/curie> a schema:Person ;
    schema:birthDate "1867-11-07"^^xsd:date ;
    schema:name "Marie Curie" .

=== RDF/XML (449 octets, 11 lignes) ===
<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:schema="https://schema.org/"
>
  <rdf:Description rdf:about="https://example.org/people/curie">
    <rdf:type rdf:resource="https://schema.org/Person"/>
   

### Interpretation : comparaison des formats

| Critere | JSON-LD | Turtle | RDF/XML | N-Triples |
|---------|---------|--------|---------|------------|
| **Lisibilite humaine** | Bonne | Excellente | Faible | Moyenne |
| **Lisibilite machine** | Excellente | Bonne | Bonne | Excellente |
| **Taille** | Moyenne | Compacte | Verbose | Verbose |
| **Integration web** | Native (JSON) | Non | Non | Non |
| **Ecosysteme dev** | Enorme (JSON) | Niche | Historique | Niche |
| **Streaming** | Non | Non | Non | Oui (ligne/ligne) |
| **Standard W3C** | 1.1 (2020) | 1.1 (2014) | 1.1 (2014) | 1.1 (2014) |

**Recommandations** :
- **JSON-LD** : integration dans les pages web, APIs, developpeurs JSON
- **Turtle** : edition humaine, ontologies, enseignement
- **RDF/XML** : systemes legacy, interoperabilite XML
- **N-Triples** : traitement en masse, streaming, tri lexicographique

---

## 8. Chargement du fichier produit dans rdflib

Pour conclure la partie pratique, chargeons le fichier `data/product.jsonld` dans un graphe rdflib et explorons-le avec SPARQL.

In [19]:
from rdflib import Graph

# Charger le fichier JSON-LD dans un graphe
g_product = Graph()
g_product.parse("data/product.jsonld", format="json-ld")

print(f"Triples charges depuis product.jsonld : {len(g_product)}")
print()

# Requete SPARQL : extraire les informations du produit
query_product = """
PREFIX schema: <https://schema.org/>

SELECT ?property ?value
WHERE {
    ?product a schema:Product .
    ?product ?property ?value .
    FILTER(!isBlank(?value))
}
ORDER BY ?property
"""

print("=== Proprietes du produit ===")
for row in g_product.query(query_product):
    prop = str(row.property).replace("https://schema.org/", "schema:")
    prop = prop.replace("http://www.w3.org/1999/02/22-rdf-syntax-ns#", "rdf:")
    print(f"  {prop:<30} {row.value}")

print()

# Requete : sujets enseignes
query_teaches = """
PREFIX schema: <https://schema.org/>

SELECT ?topic
WHERE {
    ?product a schema:Product .
    ?product schema:teaches ?topic .
}
"""

print("=== Sujets enseignes ===")
for i, row in enumerate(g_product.query(query_teaches), 1):
    print(f"  {i}. {row.topic}")

print()

# Serialiser en Turtle pour voir la structure RDF
print("=== Representation Turtle ===")
print(g_product.serialize(format="turtle"))

Triples charges depuis product.jsonld : 24

=== Proprietes du produit ===

=== Sujets enseignes ===

=== Representation Turtle ===
@prefix schema1: <http://schema.org/> .

[] a schema1:Product ;
    schema1:author [ a schema1:Person ;
            schema1:name "Jean-Sebastien Boige" ] ;
    schema1:brand [ a schema1:Brand ;
            schema1:name "CoursIA" ] ;
    schema1:description "Cours complet sur le Web Semantique, de RDF aux graphes de connaissances" ;
    schema1:educationalLevel "University" ;
    schema1:image <https://example.org/images/semantic-web-course.png> ;
    schema1:name "Semantic Web avec Python et .NET" ;
    schema1:offers [ a schema1:Offer ;
            schema1:availability "https://schema.org/InStock" ;
            schema1:price "0" ;
            schema1:priceCurrency "EUR" ;
            schema1:url <https://github.com/jsboige/CoursIA> ] ;
    schema1:teaches "GraphRAG",
        "JSON-LD et Schema.org",
        "Knowledge Graphs",
        "OWL ontologies",
   

### Interpretation

Le fichier `product.jsonld` a ete charge avec succes dans rdflib. Nous pouvons observer que :
- Le contexte `https://schema.org/` a ete correctement resolu en IRIs complets
- Les objets imbriques (brand, offers, author) sont representes comme des blank nodes avec leurs propres triples
- La propriete `teaches` genere un triple par sujet enseigne (tableau JSON -> triples multiples)

Cela demontre la puissance de JSON-LD : un document JSON lisible se transforme en un graphe RDF interrogeable.

---

## Exercices

### Exercice 1 : Creer une Schema.org Person

Creez un document JSON-LD decrivant une personne fictive avec les proprietes suivantes :
- `name`, `jobTitle`, `email`, `telephone`
- `worksFor` (une `Organization`)
- `alumniOf` (une `CollegeOrUniversity`)
- `sameAs` (liens vers profils sociaux)

Chargez-le dans rdflib et affichez les triples.

In [20]:
# Exercice 1 : Creer une Schema.org Person
# TODO : Completez le document JSON-LD ci-dessous

import json
from rdflib import Graph

person_jsonld = {
    "@context": "https://schema.org/",
    "@type": "Person",
    # TODO : Ajoutez les proprietes demandees
    # "name": "...",
    # "jobTitle": "...",
    # "email": "...",
    # "telephone": "...",
    # "worksFor": { "@type": "Organization", "name": "..." },
    # "alumniOf": { "@type": "CollegeOrUniversity", "name": "..." },
    # "sameAs": ["https://twitter.com/...", "https://linkedin.com/in/..."]
}

# Afficher le document
print(json.dumps(person_jsonld, indent=2, ensure_ascii=False))

# Charger dans rdflib et afficher les triples
# g_ex1 = Graph()
# g_ex1.parse(data=json.dumps(person_jsonld), format="json-ld")
# print(f"\nTriples : {len(g_ex1)}")
# for s, p, o in g_ex1:
#     print(f"  {s} {p} {o}")

{
  "@context": "https://schema.org/",
  "@type": "Person"
}


### Exercice 2 : Convertir du Turtle en JSON-LD

Partez du Turtle suivant et convertissez-le en JSON-LD avec rdflib :

```turtle
@prefix schema: <https://schema.org/> .
@prefix ex: <https://example.org/> .

ex:book1 a schema:Book ;
    schema:name "Le Petit Prince" ;
    schema:author ex:saint-exupery ;
    schema:datePublished "1943" ;
    schema:inLanguage "fr" .

ex:saint-exupery a schema:Person ;
    schema:name "Antoine de Saint-Exupery" .
```

In [21]:
# Exercice 2 : Convertir Turtle -> JSON-LD
from rdflib import Graph

turtle_data = """
@prefix schema: <https://schema.org/> .
@prefix ex: <https://example.org/> .

ex:book1 a schema:Book ;
    schema:name "Le Petit Prince" ;
    schema:author ex:saint-exupery ;
    schema:datePublished "1943" ;
    schema:inLanguage "fr" .

ex:saint-exupery a schema:Person ;
    schema:name "Antoine de Saint-Exupery" .
"""

# TODO : Chargez le Turtle dans un graphe rdflib
# g_ex2 = Graph()
# g_ex2.parse(data=turtle_data, format="turtle")

# TODO : Serialisez en JSON-LD
# jsonld_result = g_ex2.serialize(format="json-ld", indent=2)
# print(jsonld_result)

### Exercice 3 : Creer un Schema.org Recipe

Creez un schema `Recipe` complet pour votre plat prefere. Incluez :
- Nom, description, auteur
- Temps de preparation et cuisson (format ISO 8601 Duration : `PT30M`)
- Au moins 5 ingredients
- Au moins 3 etapes (`HowToStep`)
- Informations nutritionnelles

Chargez-le dans rdflib, puis effectuez une requete SPARQL pour extraire les ingredients.

In [22]:
# Exercice 3 : Creer un Schema.org Recipe
import json
from rdflib import Graph

# TODO : Creez votre recette en JSON-LD
my_recipe = {
    "@context": "https://schema.org/",
    "@type": "Recipe",
    # TODO : Completez avec les proprietes demandees
}

# print(json.dumps(my_recipe, indent=2, ensure_ascii=False))

# TODO : Chargez dans rdflib
# g_ex3 = Graph()
# g_ex3.parse(data=json.dumps(my_recipe), format="json-ld")

# TODO : Requete SPARQL pour extraire les ingredients
# query_ingredients = """
# PREFIX schema: <https://schema.org/>
# SELECT ?ingredient
# WHERE {
#     ?recipe a schema:Recipe .
#     ?recipe schema:recipeIngredient ?ingredient .
# }
# """
# for row in g_ex3.query(query_ingredients):
#     print(f"  - {row.ingredient}")

---

## Resume

Ce notebook a couvert les aspects essentiels de JSON-LD et Schema.org.

### Concepts cles

| Concept | Description |
|---------|-------------|
| **JSON-LD** | Format W3C qui ajoute une couche semantique au JSON via `@context` |
| **@context** | Dictionnaire de traduction entre cles JSON et IRIs RDF |
| **@id** | Identifiant de la ressource (sujet du triple) |
| **@type** | Type de la ressource (equivalent `rdf:type`) |
| **@graph** | Conteneur pour plusieurs entites dans un meme document |
| **Schema.org** | Vocabulaire collaboratif (Google, Microsoft, Yahoo) avec 800+ types |
| **Rich snippets** | Resultats enrichis dans les moteurs de recherche |

### Competences acquises

1. Comprendre la motivation et la syntaxe de JSON-LD
2. Manipuler les trois formes (compacte, etendue, aplatie)
3. Utiliser Schema.org pour decrire des entites web
4. Creer du JSON-LD avec rdflib (`g.serialize(format="json-ld")`)
5. Parser du JSON-LD et l'interroger en SPARQL
6. Effectuer des conversions round-trip entre formats
7. Appliquer les schemas web courants (Recipe, Event, FAQ, Breadcrumb)

### Pour aller plus loin

- [JSON-LD Specification (W3C)](https://www.w3.org/TR/json-ld11/)
- [JSON-LD Playground](https://json-ld.org/playground/) - testeur en ligne
- [Schema.org](https://schema.org/) - documentation des types et proprietes
- [Google Structured Data Testing Tool](https://search.google.com/test/rich-results)
- [JSON-LD Best Practices (W3C)](https://www.w3.org/TR/json-ld11-api/)

---

Le notebook suivant explore RDF 1.2 (RDF-Star), une extension majeure de RDF permettant de faire des assertions sur des assertions.

---

**Navigation** : [<< 9-SHACL](SW-9-SHACL.ipynb) | [Index](README.md) | [11-RDFStar >>](SW-11-RDFStar.ipynb)